Устанавливаем и импортируем необходимые библиотеки

In [ ]:
#Установка
!pip install folium
!pip install geopy
!pip install gmaps

In [ ]:
#Библиотеки
import numpy as np
import pandas as pd

import folium
from folium import plugins
import gmaps

from geopy.geocoders import Nominatim

import requests

import re
import json

**Функция определяет центральную координату.**



---



Функция является основной по поиску координаты центральной точки (Барицентр).

Суть функции в подсчёте средней координаты по X и Y.

In [ ]:
#Центр по координатам
def center(adresses):

  """
  Barycenter
  The essence of the function is to calculate the average X and Y coordinates.
  Output -> tuple(latitude, longitude)
  """

  list_x = []
  list_y = []

  for i in range(len(adresses)):
    list_x.append(float(adresses[i][0]))
    list_y.append(float(adresses[i][1]))

  x = np.array(list_x)
  y = np.array(list_y)
  center_point = [np.mean(x, axis=0), np.mean(y, axis=0)]

  return center_point

**Адресная книга**



---



Создаем датасет с пользователями и адресами.

Пользователи вводят данные по своей геолокации и формируется датасет данных.

In [ ]:
#Создаем пустой датасет

adress_book = pd.DataFrame()
adress_book['User_Name'] = None
adress_book['City'] = None
adress_book['Street'] = None
adress_book['House_Number'] = None
adress_book['Building'] = None
adress_book['Full_adress'] = None

adress_book['Coordinates'] = None

In [ ]:
n = 0

while True:

  """
  Users enter data on their geolocation and a dataset is formed.
  """

  n += 1
  flag = input('Добавить участника? (Do you want to add a user?) ')
  if flag == 'да' or flag == 'Да' or flag == 'Yes' or flag == 'yes':

    adress_book.loc[n, 'User_Name'] = input('Введите Ваше имя (Name):')
    adress_book.loc[n, 'City'] = input('Введите город (City):')
    adress_book.loc[n, 'Street'] = input('Введите улицу (Street):')
    adress_book.loc[n, 'House_Number'] = input('Введите номер дома (House number):')
    adress_book.loc[n, 'Building'] = input('Введите корпус (если есть) (Building):')


    if adress_book.loc[n, 'Building'] == '':
      adress_book.loc[n, 'Full_adress'] = adress_book.loc[n, 'City'] + ' ' + adress_book.loc[n, 'Street'] + ' ' + adress_book.loc[n, 'House_Number']
    else:
      adress_book.loc[n, 'Full_adress'] = adress_book.loc[n, 'City'] + ' ' + adress_book.loc[n, 'Street'] + ' ' + adress_book.loc[n, 'House_Number'] + ' ' + adress_book.loc[n, 'Building']
    print('\n')

  else:
    break

**Получаем координаты по адресам**



---



Делаем запрос в GoogleMaps по адресу и обрабатываем ответ. 

Вытягиваем координаты в теле ответа.

Добавляем координаты в датасет.

In [ ]:
for i in range(len(adress_book)):

  #Делаем запрос по адресу
  r = requests.get('https://www.google.com/maps/search/' + adress_book.loc[i+1, 'Full_adress'])

  #Регулярными выражениями находим координаты в тексте запроса
  coord = re.findall(r'center=.+?&', r.text)[0][7:-1]

  #Добавляем координаты в датасет
  latitude = re.findall(r'.+%', coord)[0][:-1]
  longitude = re.findall(r'C.+', coord)[0][1:]

  adress_book.loc[i+1, 'Coordinates'] = (latitude, longitude)

In [ ]:
#Итоговый датасет с координатами
#Final dataset with coordinates

adress_book

**Находим центральную точку**



---




Центральная точка (Барицентр) - это среднее арифметическое положений всех точек координат пользователей.

In [ ]:
#Центральная точка (Барицентр)

center_point = center(list(adress_book['Coordinates']))
center_point

**Ищем ближайший бар** (в стадии разработки)



---



Находим ближайший бар к ценртральной точке

In [ ]:
#Адрес поисковика бара

# bar = requests.get('https://www.google.ru/maps/search/bar/@' + f'{center_point[0]}' + ','  + f'{center_point[1]}' + ',15z' )

In [ ]:
#Название ближайшего бара

# bar_name = re.findall(r'".+Owner', bar.text[40000:50000])[0][-50:]
# bar_name = re.findall(r',\\".+',bar_name)[0][3:-7]
# bar_name

In [ ]:
#ДОСТАЕМ КООРДИНАТЫ БАРА или адрес

# coor_bar = bar.text[40000:50000]

**Итоговая часть**



---



Наносим на карту все необходимые метки.

In [ ]:
#Наносим координаты на карту

m = folium.Map(location=[center_point[0], center_point[1]], zoom_start=10)

#Цвет маркеров
icon_center = folium.Icon(color="green")

#Маркер центра
folium.Marker((center_point[0], center_point[1]), tooltip="Центр", icon=icon_center).add_to(m)

#Маркеры адресов
for i in range(len(adress_book)):
  folium.Marker((float(adress_book.loc[i+1, 'Coordinates'][0]), float(adress_book.loc[i+1, 'Coordinates'][1])), tooltip=adress_book.loc[i+1, 'User_Name']).add_to(m)

m